<a href="https://colab.research.google.com/github/darkknight314/Dog-Cat-Classification/blob/master/Resnet18%2BDense_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
cd /content/drive/My Drive/Dataset224

/content/drive/.shortcut-targets-by-id/1tMtDHOL-zdhTUWXJ0P8RKeeAp7ccVdKb/Dataset224


In [0]:
import torch
import numpy as np
from torchvision.models import resnet18
import torch.nn as nn
from sklearn.utils import shuffle
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms

In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [0]:
X = np.load("./images.npz", allow_pickle=True)['arr_0']
y = np.load("./species.npz", allow_pickle=True)['arr_0']
y = y-1
X,y = shuffle(X,y)

In [0]:
TRAIN = int(0.8*y.shape[0])
BATCH_SIZE = 16
EPOCH = 10

In [11]:
resmodel = resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [0]:
class ResClass(nn.Module):
    def __init__(self, resmodel):
        super(ResClass, self).__init__()
        self.resout = resmodel
        self.output = nn.Linear(1000,2)
    def forward(self, input):
        resout = self.resout(input)
        output = self.output(resout)
        return output 

In [0]:
model = ResClass(resmodel)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

In [0]:
class MyDataset(Dataset):
    def __init__(self,X,y, transforms):
        self.X = X
        self.y = y
        self.transforms = transforms
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        img, label = self.X[idx], self.y[idx]
        if self.transforms is not None:
            img = self.transforms(img)
        return img, label

In [0]:
apply_transforms = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.ColorJitter(brightness = 0.5, contrast = 0.5, saturation = 0.5, hue = 0.5),
                    transforms.RandomGrayscale(0.1),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.RandomVerticalFlip(p=0.5),
                    transforms.RandomAffine(30),
                    transforms.ToTensor(),
                    ])
train_dataset = MyDataset(X[:TRAIN],y[:TRAIN],apply_transforms)
test_dataset = MyDataset(X[TRAIN:],y[TRAIN:], transforms.Compose([transforms.ToTensor()]))
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [0]:
def calculate_accuracy(output, target):
  return sum(torch.max(output.to('cpu'),1).indices==target.to('cpu'))  

In [17]:
loss_list = []
for epoch in range(EPOCH):
    running_loss = 0.0
    running_accuracy = 0.0
    for idx, (img, label) in enumerate(train_data):
        optimizer.zero_grad()
        img = img.to(device)
        label = label.long().to(device)
        out = model(img)
        loss = criterion(out, label)
        loss.backward()                                 
        optimizer.step()
        running_loss+=loss.item()
        running_accuracy += calculate_accuracy(out, label)
    loss_list.append(running_loss)
    print('Epoch: ', epoch, '| train loss: %.4f' % running_loss, '| train accuracy: %.2f' % (100*(running_accuracy.item())/len(train_dataset))) 

Epoch:  0 | train loss: 124.1930 | train accuracy: 85.12
Epoch:  1 | train loss: 74.5240 | train accuracy: 91.63
Epoch:  2 | train loss: 66.2216 | train accuracy: 92.63
Epoch:  3 | train loss: 55.9280 | train accuracy: 94.01
Epoch:  4 | train loss: 49.2610 | train accuracy: 94.74
Epoch:  5 | train loss: 46.9357 | train accuracy: 95.01
Epoch:  6 | train loss: 44.0021 | train accuracy: 95.21
Epoch:  7 | train loss: 38.3684 | train accuracy: 95.87
Epoch:  8 | train loss: 36.2530 | train accuracy: 96.28
Epoch:  9 | train loss: 33.2066 | train accuracy: 96.92


In [18]:
with torch.no_grad():
    test_accuracy = 0.0
    base = 0
    for idx, (img, label) in enumerate(test_data):
        img = img.to(device)
        base += sum(label).item()
        label = label.long().to(device)
        out = model(img)
        test_accuracy += calculate_accuracy(out,label)
    print("Test accuracy: ",(100*test_accuracy.item()/len(test_dataset)),"Base: ",(100*base/len(test_dataset)))

Test accuracy:  97.90257104194858 Base:  66.84709066305818
